# Resume NER
## Extract Information from Resumes using NER (Named Entity Recognition)

### Part 2 - NER with Spacy
In this second part of the challenge, we will be using the preprocessed data from part one to start training NER models. We will be using spacy (https://spacy.io/) here to "get our feet wet" with NER, as training spacy can be reasonably done on our laptops and does not yet necessarily require a GPU. Spacy is a powerful, effective, and resource-efficient NLP library - It might surprise us with its performance on the challenge!

We will run spacy's pretrained models on our data to get a feel for NER, and then we will perform some additional preprocessing on our data before we start training our own NER model using the labelled entities we have identified in part one. 
We will also explore evaluation metrics for NER, and decide how we want to quantify the performance of our trained models. 

* *If you need help setting up python or running this notebook, please get help from the  assistants to the professor*
* *It might be helpful to try your code out first in a python ide like pycharm before copying it an running it here in this notebook*
* *For solving the programming tasks, use the python reference linked here (Help->Python Reference) as well as Web-searches.* 

##### Reload preprocessed data
Here, we will load the data we saved in part one and save it to a variable. Provide code below to load the data and store it as a list in a variable. (Hint - use 'open' and the json module)

In [103]:
## import json module
import json
path = "data/converted_resumes.json"
## TODO open file load as json and store in "resumes" variable
resumes = json.load(open(path, "r", encoding="utf-8"))
## TODO print length of loaded resumes list to be sure everything ok
print(len(resumes))

690


In [104]:
import platform
print(platform.python_version())

3.7.3


##### Take Spacy for a spin
Before we train our own NER model to recognize the resume-specific entities we want to capture, let's see how spacy's pretrained NER models do on our data. These pretrained models can't recognize our entities yet, but let's see how they do. Run the next code block to load spacy's English language model 


In [105]:
import spacy
nlp = spacy.load('en')
print(nlp)

Now we get the EntityRecognizer in the loaded nlp pipeline and display the labels it supports

In [106]:
ner = nlp.get_pipe('ner')
labels = ner.labels
print(labels)

('ORDINAL', 'LAW', 'MONEY', 'DATE', 'ORG', 'TIME', 'EVENT', 'CARDINAL', 'PRODUCT', 'QUANTITY', 'LANGUAGE', 'PERSON', 'LOC', 'FAC', 'NORP', 'WORK_OF_ART', 'GPE', 'PERCENT')


##### Question: What do the 'GPE', 'FAC' and 'NORP' labels stand for? (Tipp: use either the spacy.explain method, or google the spacy.io api docs) 
*Answer here*

In [107]:
### TODO  if you choose to use spacy's 'explain' method to get the answer to the question above, provide your code here
## print description of entities using spacy explain
print("GPE: ",spacy.explain("GPE"))
print("FAC: ",spacy.explain("FAC"))
print("NORP: ",spacy.explain("NORP"))

GPE:  Countries, cities, states
FAC:  Buildings, airports, highways, bridges, etc.
NORP:  Nationalities or religious or political groups


As we can see, the entities are different than the entities we will train our custom model on. 
##### Question: what entities do you think this model will find in an example resume?
Nationality, Buildings of institutions the individual worked/studied at, cities they lived/worked in

Now we will work with one of our resumes, and get spacy to tell us what entities it recognizes. Complete the code block below to get a single resume text out of our resume list. 

In [108]:
### TODO get a single resume text and print it out
restxt = resumes[0][0]
## print it out, removing extraneous spaces
print("\n".join(restxt.split('\n\n')))

Afreen Jamadar
Active member of IIIT Committee in Third year
Sangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
I wish to use my knowledge, skills and conceptual understanding to create excellent team
environments and work consistently achieving organization objectives believes in taking initiative
and work to excellence in my work.
WORK EXPERIENCE
Active member of IIIT Committee in Third year
Cisco Networking -  Kanpur, Uttar Pradesh
organized by Techkriti IIT Kanpur and Azure Skynet.
PERSONALLITY TRAITS:
• Quick learning ability
• hard working
EDUCATION
PG-DAC
CDAC ACTS
2017
Bachelor of Engg in Information Technology
Shivaji University Kolhapur -  Kolhapur, Maharashtra
2016
SKILLS
Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT
ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)
ADDITIONAL INFORMATION
TECHNICAL SKILLS:
• Programming Languages: C, C++, Java, .net, php.
• Web Designing: HTML, XML

Extracting entities with spacy is easy with a pretrained model. We simply call the model (here 'nlp') with our text to get a spacy Document. See https://spacy.io/api/doc for more detail. 

Execute the code below to process the resume txt.

In [109]:
doc = nlp(restxt)

The doc object has a list of entities predicted by spacy 'ents'. We would like to loop through all of these entities and print their label and associated text to see what spacy predicted for this resume.

Complete the code below to do this. You will probably need to google the spacy api docs to find the solution (Tipp: look for 'Doc.ents'). Also, trying code in your ide (for example pycharm) before copying it here might help with exploring and debugging to find the solution. 

In [110]:
##TODO loop through the doc's entities, and print the label and text for each entity found. 
for x in doc.ents:
    print(x.label_, x.text)

GPE Jamadar
ORG IIIT Committee
DATE Third year
ORG Maharashtra
ORG IIIT Committee
DATE Third year
PERSON Cisco Networking -  Kanpur
ORG Uttar Pradesh
ORG Techkriti IIT Kanpur
PERSON Azure Skynet
PERSON Quick
ORG CDAC
DATE 2017
GPE Kolhapur
ORG Maharashtra
DATE 2016
ORG HTML
DATE Less than 1 year
PERSON Linux
DATE Less than 1 year
ORG MICROSOFT
PERSON ACCESS
PERSON C++
PERSON Java
ORG XML
ORG Operating Systems
PERSON Windows Server
DATE 2003
GPE Linux
ORG MS Access
ORG MS SQL
DATE 2008
PRODUCT Oracle 10
PRODUCT MySql


##### Questions: What is your first impression of spacy's NER based on the results above? Does it seem accurate/powerfull?
##### Does it make many mistakes? Do some entity types seem more accurate than others? 
In some instances, like PRODUCT Oracle 10 or DATE 2008 it does seem accurate.
It still struggles in some cases, like GPE Jamadar.

Now as a comparison, we will list the entities contained in the resume's original annotated training data (remember, the existing annotations were created by a human-annotator, and not predicted by a machine like the entities predicted above) 

Complete the code below to do the following: 
* Access the 'entities' list of the example resume you chose, loop through the entities and print them out. 
* *Tip: one entity in the list is a tuple with the following structure: (12,1222,"label") where the first element is the start index of the entity in the resume text, the second element is the end index, and the third element is the label.
* Use this Tip to print out a formatted list of entities 



In [111]:
##TODO print original entities for one resume
res = resumes[0]
restext = res[0] 
labeled_ents = res[1]["entities"]
## TDOD print out formatted list of entity labels and text
for ent in labeled_ents:
    print("{} {}".format(ent[2], restext[ent[0]:ent[1]]))

Email Address indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
Links https://www.indeed.com/r/Afreen-Jamadar/8baf379b705e37c6?isid=rex-download&ikw=download-top&co=IN
Skills Database (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT
ACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)

ADDITIONAL INFORMATION

TECHNICAL SKILLS:

• Programming Languages: C, C++, Java, .net, php.
• Web Designing: HTML, XML
• Operating Systems: Windows […] Windows Server 2003, Linux.
• Database: MS Access, MS SQL Server 2008, Oracle 10g, MySql.
Graduation Year 2016
College Name Shivaji University Kolhapur 
Degree Bachelor of Engg in Information Technology
Graduation Year 2017

College Name CDAC ACTS
Degree PG-DAC
Companies worked at Cisco Networking
Email Address indeed.com/r/Afreen-Jamadar/8baf379b705e37c6
Location Sangli
Name Afreen Jamadar


As we already know, the annotated entities in the training data are different than the entities spacy can recognize with it's pretrainied models, so we need to train a custom NER model. We will get started with that now. 

In [112]:
print(len(test))

796446


##### Prepare Training Data for NER model training
We need to do some more preprocessing of our training data before we can train our model.

Remember the entity labels you chose in part 1 of the challenge? We will be training a model to predict those entities.
As a first step, we will gather all resumes that contain at least one training annotation for those entities.

Complete and execute the code below to gather your training data. 

In [113]:
##TODO Store the entity labels you want to train for as array in chosen_entity_labels
chosen_entity_labels = ["Skills", "Location", "Degree"]
## this method gathers all resumes which have all of the chosen entites above.
def gather_candidates(dataset,entity_labels):
    candidates = list()
    for resume in dataset:
        res_ent_labels = list(zip(*resume[1]["entities"]))[2]
        if set(entity_labels).issubset(res_ent_labels):
            candidates.append(resume)
    return candidates
## TODO use the gather candidates methods and store result in training_data variable
training_data = gather_candidates(resumes, chosen_entity_labels)
print("Gathered {} training examples".format(len(training_data)))

Gathered 447 training examples


Now we have those training examples which contain the entities we are interested in. Do you have at least a few hundred examples? If not, you might need to re-think the entities you chose or try just one or two of them and re-run the notebooks. It is important that we have several hundred examples for training (e.g. more than 200. 3-500 is better). 

##### Remove other entity annotations from training data
Now that we have our training data, we want to remove all but relevant (chosen) entity annotations from this data, so that the model we train will only train for our entities. Complete and execute the code below to do this. 

In [114]:
## filter all annotation based on filter list
def filter_ents(ents, filter):
    filtered = [ent for ent in ents if ent[2] in filter]
    return filtered

## TODO use method above to remove all but relevant (chosen) entity annotations and store in X variable 
X = resumes
for res in X:
    res[1]["entities"] = filter_ents(res[1]["entities"], chosen_entity_labels)

print(X[0])

['Afreen Jamadar\nActive member of IIIT Committee in Third year\n\nSangli, Maharashtra - Email me on Indeed: indeed.com/r/Afreen-Jamadar/8baf379b705e37c6\n\nI wish to use my knowledge, skills and conceptual understanding to create excellent team\nenvironments and work consistently achieving organization objectives believes in taking initiative\nand work to excellence in my work.\n\nWORK EXPERIENCE\n\nActive member of IIIT Committee in Third year\n\nCisco Networking -  Kanpur, Uttar Pradesh\n\norganized by Techkriti IIT Kanpur and Azure Skynet.\nPERSONALLITY TRAITS:\n• Quick learning ability\n• hard working\n\nEDUCATION\n\nPG-DAC\n\nCDAC ACTS\n\n2017\n\nBachelor of Engg in Information Technology\n\nShivaji University Kolhapur -  Kolhapur, Maharashtra\n\n2016\n\nSKILLS\n\nDatabase (Less than 1 year), HTML (Less than 1 year), Linux. (Less than 1 year), MICROSOFT\nACCESS (Less than 1 year), MICROSOFT WINDOWS (Less than 1 year)\n\nADDITIONAL INFORMATION\n\nTECHNICAL SKILLS:\n\n• Programming

##### Remove resumes that cause errors in spacy
Depending on what entities you chose, some of the resumes might cause errors in spacy. We don't need to get into details as to why, suffice to say it has to do with whitespace and syntax in the entity annotations. If these resumes are not removed from our training data, spacy will throw an exception during training, so we need to remove them first. 

We will use the remove_bad_data function below to do this. This function does the following:
* calls train_spacy_ner with debug=True and n_iter=1. This causes spacy to process the documents one-by-one, and gather the documents that throw an exception in a list of "bad docs" which it returns. 
* You will complete the function to remove any baddocs (returned by remove_bad_data) from your training data list. 

You may or may not have any bad documents depending on the entities you chose. In any case, there should not be more than a dozen or so bad docs.  

In [115]:
from spacy_train_resume_ner import train_spacy_ner

def remove_bad_data(training_data):
    print()
    model, baddocs = train_spacy_ner(training_data, debug=True, n_iter=1)
    ## training data is list of lists with each list containing a text and annotations
    ## baddocs is a set of strings/resume texts.
    ## TODO complete implementation to filter bad docs and store filter result (good docs) in filtered variable
    filtered = [data for data in training_data if data[0] not in baddocs]
    print("Unfiltered training data size: ",len(training_data))
    print("Filtered training data size: ", len(filtered))
    print("Bad data size: ", len(baddocs))
    return filtered

## call remove method. It may take a few minutes for the method to complete.
## you will know it is complete when the print output above. 
X = remove_bad_data(X)
print("Done")


Created blank 'en' model
Exception thrown when processing doc:
('Deepika S\nTest Engineer - Infosys Ltd\n\n- Email me on Indeed: indeed.com/r/Deepika-S/1b4436206cf5871b\n\nWORK EXPERIENCE\n\nTest Engineer\n\nInfosys Ltd -  Chennai, Tamil Nadu -\n\nApril 2017 to Present\n\nTesting Executive, Infosys Ltd, Chennai\n\nE-publisher -  Coimbatore, Tamil Nadu -\n\nMay 2013 to July 2014\n\nSeptember 2014 - March 2017)\n\nTest Management Tool\n\nHP ALM -\n\n2012 to 2012\n\n• Domain: Retail, Banking\n• Utilities: Putty, WinScp.\n• Operating Systems: Windows XP/7/8\n\nPROFESSIONAL EXPOSURE \nSoftware Testing Projects\n\nCiti Bank -\n\n2005 to 2005\n\nDescription of the Project: The objective of the project is to track the Gifts and Entertainment\nprovided to or received from client or potential client or anyone else whom citi does business.\nCGE it is a web based application.\nAnd the project uses separate tool IFW for reporting.\nTesting Management Tool: ALM for tracking defect\nDuration: Nov 20

Exception thrown when processing doc:
("Jitendra Razdan\nBusiness Development - Shapoor ji Pallonji Group Company\n\nMumbai, Maharashtra - Email me on Indeed: indeed.com/r/Jitendra-Razdan/66b1e69aff1842bd\n\n• Dynamic professional with more than 7 years of work experience in Sales and Customer\nService to Banking, Government & Corporate Sector.\n• Proficient in managing day to day sales activities, marketing and client servicing activities.\n• Ability to develop and maintain relationship with key decision makers and resolving\ncritical problems.\n• Ability to think innovatively and grasp key concepts quickly..\n• Effective communicator with an ability to deal effectively with various departments\n\nWORK EXPERIENCE\n\nBusiness Development\n\nShapoor ji Pallonji Group Company -  Mumbai, Maharashtra -\n\nJuly 2011 to Present\n\n2011.\n\n(Areas: Mumbai, Delhi, NCR, Punjab, Himachal Pradesh, Haryana and J&K)\n\nArea of Expertise & Exposure\nKey Account Management:\n• Interface with individu

##### Question: How many bad docs did you have? What is the size of your new (filtered) training data
4 bad docs.

The new filtered training data contains 685 entries

##### Train/Test Split
Now before we train our model, we have to split our available training data into training and test sets. Splitting our data into train and test (or holdout) datasets is a fundamental technique in machine learning, and essential to avoid the problem of overfitting.
Before we go on, you should get a grasp of how train/test split helps us avoid overfitting. Please take the time now to do a quick web search on the topic. There are many resources available. You should search for "train test validation overfitting" or some subset of those terms.

Here are a few articles to start with:
* https://machinelearningmastery.com/a-simple-intuition-for-overfitting/
* https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation (Note: you are free to install scikit learn and use the train_test_split method documented here, but it is not necessary. It is the concept that is important)

##### Question: What is overfitting and how does doing a train/test split help us avoid overfitting when training our models? Please answer in your own words. 
*Answer here*

Now that we understand why we do a train/test split, we will write some code that splits our data into train and test sets. Usually we want around 70-80% of the data for train, and the rest for test. 
##### TODO: Complete the code below to create a train and test dataset

In [116]:
##TODO complete the implementation  of the train test split function below
def train_test_split(X,train_percent):
    train_size = int(len(X)*train_percent)
    train = X[:train_size]
    test = X[train_size:]
    return train,test
## TODO chose train size percent and call train test split, storing results in "train" and "test" variables.
train,test = train_test_split(X, 0.8)
## TODO use python assert to assert that the size of train and test sets add up to the size of all the data 
assert len(train) + len(test) == len(X), str(len(train) + len(test)) + " not equal to " + str(len(X))

print("Done")

Done


In [117]:
a = train
b = test

In [140]:
train = a
test = b

##### Train a spacy ner model with our training data
OK, now it is (finally) time to train our own custom NER model using spacy. Because our training data has been preprocessed to only include annotations for the entities we are interested in, the model will only be able to predict/extract those entities. 
*Depending on your computer, this step may take a while.* Training 20 epochs (iterations) using 480 training examples takes around 10 minutes on my machine (core i7 CPU). You will see output like *Losses {'ner':2342.23342342}* after each epoch/iteration. The default number of iterations is 20, so you will see this output 20 times. When this step is done, we will use the trained ner model to perform predictions on our test data in our test set.  

In [141]:
## run this code to train a ner model using spacy
custom_nlp,_= train_spacy_ner(train,n_iter=20)
print("Done")

Created blank 'en' model
Losses {'ner': 39787.608389427216}
Losses {'ner': 23052.121510999346}
Losses {'ner': 25192.887486392676}
Losses {'ner': 20446.425688700532}
Losses {'ner': 17197.79749451209}
Losses {'ner': 18330.22665392835}
Losses {'ner': 16652.33207899807}
Losses {'ner': 13741.162099086534}
Losses {'ner': 13193.302576137492}
Losses {'ner': 13462.963241381454}
Losses {'ner': 13287.69087289413}
Losses {'ner': 13032.805074130327}
Losses {'ner': 12006.593832792587}
Losses {'ner': 15250.591541561884}
Losses {'ner': 11949.649656931713}
Losses {'ner': 13377.376593731165}
Losses {'ner': 11975.005670584706}
Losses {'ner': 11975.653999915863}
Losses {'ner': 11618.249421906483}
Losses {'ner': 11118.741820057858}
Done


In [ ]:
a = train
b = test

In [ ]:
train = a
test = b

##### Inspect NER predictions on one sample resume
Now that we have a trained model, let's see how it works on one of our resumes. 

In [142]:
## TODO fetch one resume out of our test dataset and store to the "resume" variable
resume = test[0]
## TODO create a spacy doc out of the resume using our trained model and save to the "doc" variable 
doc = custom_nlp(resume[0])

Now we will output the predicted entities and the existing annotated entities in that doc

137


In [143]:
## TODO output label and text of predicted entities (in "ents" variable of the spacy doc created above)
print("PREDICTED:")
for ent in doc.ents:
    print(ent.label_, ent.text)
## TODO output labeled entities (in "entities" dictionary of resume)
print("LABELED:")
for ent in reversed(resume[1]["entities"]):
    print(ent[2])    


PREDICTED:
Location Delhi
Degree MCA
Degree BCA
Location Pune
Location New Delhi
Location New Delhi
Skills ACCESS
Skills BUYING
Skills BUYING/
Skills CSS
Skills DATABASE
LABELED:
Location
Location
Degree
Degree
Location
Location
Location
Location
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills
Skills


#### Evaluation Metrics for NER
Now that we can predict entities using our trained model, we can compare our predictions with the original annotations in our training data to evaluate how well our model performs for our task. The original annotations have been annotated manually by human annotators, and represent a "Gold Standard" against which we can compare our predictions. 

For most classification tasks, the most common evaluation metrics are:
* accuracy
* precision
* recall
* f1 score

In order to understand these metrics, we need to understand the following concepts:
* True positives - How many of the predicted entities are "true" according to the Gold Standard? (training annotation) 
* True negatives - How many entities did the model not predict which are actually not entities according to the Gold Standard?
* False positives - How many entities did the model predict which are NOT entities according to the Gold Standard?  
* False negatives - How many entities did the model "miss" - e.g. did not recognize as entities which are entities according to the Gold Standard? 

Before we go on, it is important that you understand true/false positives/negatives as well as the evaluation metrics above. Take some time now to research the web in order to find answers to the following questions:

##### Question: How are the evaluation metrics above defined in the context of evaluating Machine Learning models? How do they relate to True/False Positives/Negatives above? Please provide an intuitive description as well as the mathmatical formula for each metric. 
*Answers here*


##### Calculating Metrics based on token-level annotations or full entity-level. 
The concepts above are our first step toward understanding how to evaluate our model effectively. However, in NER, we need to take into account that we can calculate our metrics either based on all tokens (words) found in the document, or only on the entities found in the document.  

##### Token-Level evaluation. 
Token level evaluation evaluates how accurately did the model tag *each individual word/token* in the input. In order to understand this, we need to understand something called the "BILUO" Scheme (or BILOU or BIO). The spacy docs have a good reference. Please read and familiarize yourself with BILUO. 

https://spacy.io/api/annotation#biluo

Up to now, we have not been working with the BILUO scheme, but with "offsets" (for example: (112,150,"Email") - which says there is an "Email" entity between positions 112 and 150 in the text). We would like to be able to evaluate our models on a token-level using BILUO - so we need to convert our data to BILUO. Fortunately, Spacy provides a helper method to do this for us.

*Execute the code below to see how our "Gold Standard" and predictions for our example doc above look in BILUO scheme.* 
Note: some of the lines might be ommited for display purposes. 

In [144]:
from spacy.gold import biluo_tags_from_offsets
import pandas as pd
from IPython.display import display, HTML

## returns a pandas dataframe with tokens, prediction, and true (Gold Standard) annotations of tokens
def make_bilou_df(nlp,resume):
    """
    param nlp - a trained spacy model
    param resume - a resume from our train or test set
    """
    doc = nlp(resume[0])
    bilou_ents_predicted = biluo_tags_from_offsets(doc, [(ent.start_char,ent.end_char,ent.label_)for ent in doc.ents])
    bilou_ents_true = biluo_tags_from_offsets(doc,
                                                   [(ent[0], ent[1], ent[2]) for ent in resume[1]["entities"]])

    
    doc_tokens = [tok.text for tok in doc]
    bilou_df = pd.DataFrame()
    bilou_df["Tokens"] =doc_tokens
    bilou_df["Tokens"] = bilou_df["Tokens"].str.replace("\\s+","") 
    bilou_df["Predicted"] = bilou_ents_predicted
    bilou_df["True"] = bilou_ents_true
    return bilou_df

## TODO call method above with a resume from test set and store result in bilou_df variable.
bilou_df = make_bilou_df(custom_nlp, resumes[1])
display(bilou_df)  
print(1)

,Tokens,Predicted,True
0,Arun,O,O
1,Elumalai,O,O
2,,O,O
3,QA,O,O
4,Tester,O,O
5,,O,O
6,Chennai,U-Location,U-Location
7,",",O,O
8,Tamil,O,O
9,Nadu,O,O


1


In [123]:
print(len(test))

137


Based on this output, it should be very easy to calculate a token-level accuracy. We simply compare the "Predicted" to "True" columns and calculate what percentage are the same. 

In [145]:
## TODO bilou_df is a pandas dataframe. Use pandas dataframe api to get a subset where predicted and true are the same. 
same_df = bilou_df.loc[bilou_df['Predicted'] == bilou_df["True"]]
## accuracy is the length of this subset divided by the length of bilou_df
accuracy = len(same_df)/len(bilou_df)
print("Accuracy on one resume: ",accuracy)

Accuracy on one resume:  0.9771573604060914


The accuracy might seem pretty good... if it is not 100%, then let's print out those tokens where the model predicted something different than the gold standard by running the code below. 

Note - if your score on one doc is 100%, pick another document and re-run the last few cells above. 

In [146]:
## TODO find all rows in bilou_df where "Predicted" not equal to "True" column. 
diff_df = bilou_df.loc[bilou_df['Predicted'] != bilou_df["True"]]
display(diff_df)

,Tokens,Predicted,True
359,Assurance,I-Skills,L-Skills
360,,I-Skills,O
361,ADDITIONAL,I-Skills,O
362,INFORMATION,I-Skills,O
363,,I-Skills,O
364,TECHNICAL,I-Skills,O
365,SKILLS,I-Skills,O
366,,I-Skills,O
367,•,I-Skills,B-Skills


Now let's calculate the accuracy on all our test resumes and average them for an accuracy score. 

Please complete the code below to report an accuracy score on our test resumes

In [126]:
print(len(test))

137


In [147]:
import numpy as np
doc_accuracy = []
for res in test:
    ## TODO calculate accurac for each 'res' and append to doc_accuracy list
    res_df = make_bilou_df(custom_nlp, res)
    res_same_df = res_df.loc[res_df['Predicted'] == res_df["True"]]
    doc_accuracy.append(len(res_same_df)/len(res_df))


## TODO calculate mean/average of doc_accuracy (Tip: use numpy!)
total_acc = np.mean(doc_accuracy)
print("Accuracy: ",total_acc)

    
    

Accuracy:  0.9086417779005374


In [128]:
print(len(test))

137


##### Question: how does the model perform on token-level accuracy? What did it miss? In those cases where the predictions didn't match the gold standard, were the predictions plausible or just "spurious" (wrong)? 
*Answer here* 

##### Question: What might the advantages and disadvantages be of calculating accuracy on token-level? Hint: think about a document with 1000 tokens where only 10 tokens are annotated as entities. What might the accuracy be on such a document?  

##### Entity-Level evaluation #####
Another method of evaluating the performance of our NER model is to calculate metrics not on token-level, but on entity level. There is a good blog article that describes this method. 

http://www.davidsbatista.net/blog/2018/05/09/Named_Entity_Evaluation/

The article goes into some detail, the most important part is the scenarios described in the section "Comparing NER system output and golden standard". 

##### Question: how do the first 3 scenarios described in the section "Comparing NER system output and golden standard" correlate to  true/false positives/negatives? 
*Answer here* 

##### Precision, Recall, F1 #####

Now we would like to calculate precision, recall, and f1 for each entity type we are interested in (our chosen entities). To do this, we need to understand the formulas for each. A good article for this is https://skymind.ai/wiki/accuracy-precision-recall-f1. 

##### Question: how can we calculate precision, recall and f1 score based on the information above? Please provide the formulas for each #####
*Answer here*

Now supply code below which calculates precision and recall and F1 on our test data for each entity type we are interested in. 



In [148]:
## TODO cycle through chosen_entity_labels and calculate metrics for each entity using test data
data = []
for label in chosen_entity_labels:
    ## variables to store results for all resumes for one entity type
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    for tres in test:
        
        ## use make_bilou_df on each resume in our test set, and calculate for each entity true and false positives,
        ## and false negatives. 
        tres_df = make_bilou_df(custom_nlp,tres)
        ## calculate true false positives and false negatives for each resume
        tp = len(tres_df.loc[tres_df["True"].str.contains(label)].loc[tres_df["Predicted"] == tres_df["True"]])
        fp = len(tres_df.loc[tres_df["True"] == "O"].loc[tres_df["Predicted"].str.contains(label)])
        fn = len(tres_df.loc[tres_df["True"].str.contains(label)].loc[tres_df["Predicted"] == "O"])

        ## aggregate result for each resume to totals
        true_positives += tp
        false_positives += fp
        false_negatives += fn
    
    print("For label '{}' tp: {} fp: {} fn: {}".format(label,true_positives,false_positives,false_negatives))
    
    ## TODO Use the formulas you learned to calculate metrics and print them out        
    precision = true_positives/(true_positives+false_positives)
    recall = true_positives/(true_positives+false_negatives)
    f1 = 2 * (precision*recall)/(precision+recall)
    #print("Precision: ",precision)
    #print("Recall: ",recall)
    #print("F1: ",f1)
    row = [precision,recall,f1]
    data.append(row)

## make pandas dataframe with metrics data. Use the chosen entity labels as an index, and the metric names as columns. 
metric_df = pd.DataFrame(data, columns = ['precision', 'recall', "f1"], index = chosen_entity_labels) 
display(metric_df)

For label 'Skills' tp: 977 fp: 4367 fn: 1044
For label 'Location' tp: 333 fp: 150 fn: 99
For label 'Degree' tp: 458 fp: 175 fn: 41


,precision,recall,f1
Skills,0.182822,0.483424,0.265309
Location,0.689441,0.770833,0.727869
Degree,0.723539,0.917836,0.809187


Now we compute an average score for each metric. 

In [130]:
## TODO compute average metrics And print them out. Use pandas dataframe "mean" method to do this
display(metric_df.mean())

precision    0.546726
recall       0.701558
f1           0.600510
dtype: float64

##### Question: how do the average metrics here (computed on entity-level) compare to the token-level accuracy score above? Which metric(s) would you prefer to use to evaluate the quality of your model? Why? 

We are almost Done with part II! We just need to save our BILUO training data for reuse in Part III. 
For part III we are using flair and loading our data from a .csv file into a flair "Corpus". This is described here:

https://github.com/zalandoresearch/flair/blob/master/resources/docs/TUTORIAL_6_CORPUS.md#reading-your-own-sequence-labeling-dataset

We need to create "train" and "test" .csv files using our train and test dataset which corresponds to the format described above. This format is one line containing (minimally) a text token and a NER Tag. These should be separated by whitespace. 

##### "Sentences" #####
Flair works with "Sentences" which is a list of tokens. If we simply write out our csv with one line for every token in our dataset, we will have 1 giant sentence with many thousands of words.. This is not what we want. 
We would like to partition our data so that we have a list of "Sentences" - corresponding to our intuition for a sentence - a sequence of words that belong together and is not all to long, usually separated by some punctuation. 
When we create our .csv strings/files, we need to do so so that they represent a list of sentences, each sentence consisting of a list of tokens/tags (each token/tag being one line in our csv). 

***To do this, create a blank newline after each sentence.*** 

How you split your resume data into sentences is your decision however.  

Some starter code is provided below. Complete it to create two strings which you will save as .csv. Each file will be a (long) list of token/tag lines, with sentences separated by newlines. 

In [132]:
print(len(train))

print(len(test))

548
137


In [149]:
def bilou_for_flair(nlp, train, test):
    """
    make .csv strings from train and test for use in flair
    """    
    print("Make bilou dfs")
    ## makes a list of pandas dataframes, one for each resume. 
    training_data_as_bilou = [make_bilou_df(nlp,res) for res in train]
    test_data_as_bilou = [make_bilou_df(nlp,res) for res in test]
    print("Done!")
    ## strings to return
    
    training_file = ""
    test_file = ""
    for idx,df in enumerate(training_data_as_bilou):
        
        ## TODO - remove unwanted whitespace and/or newline token rows from dataframe
        df = df[~df["Tokens"].isin(["","\n"," "])]
        ## TODO - insert newlines after each "sentence" 
        df.loc[df["Tokens"] == ".", "Tokens"] = "\n"
        df.drop(columns=["Predicted"])
       
        ## You can use df.to_csv below to create a csv for your data
        as_csv = df.to_csv(None,sep=" ",encoding="utf-8",index=False,header=False,line_terminator="\n")
        
        training_file += as_csv + ("\n") 
    for idx, df in enumerate(test_data_as_bilou):
        ## TODO - remove unwanted whitespace and/or newline token rows from dataframe
        df = df[~df["Tokens"].isin(["","\n"])]
        ## TODO - insert newlines after each "sentence" 
        df.loc[df["Tokens"] == ".", "Tokens"] = "\n"
        
        ## You can use df.to_csv below to create a csv for your data
        as_csv = df.to_csv(None,sep=" ",encoding="utf-8",index=False,header=False,line_terminator="\n")
        
        test_file += as_csv + ("\n") 
        df.drop(columns=["Predicted"])
        
    return training_file,test_file

training,test = bilou_for_flair(custom_nlp,train,test)
## TODO save training and test as .csv (wherever you want)
with open("data/bilou_training.csv", "+w") as f:
    f.write(training)
with open("data/bilou_test.csv", "+w") as f:
    f.write(test)

print("Fertig")

Make bilou dfs
Done!


/home/anton/Desktop/uni/Resume_NER/nerenv/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/anton/Desktop/uni/Resume_NER/nerenv/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/anton/Desktop/uni/Resume_NER/nerenv/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fertig


In [96]:
print(len(test))

796446


Now let's load the data we persisted with flair before we go on

In [137]:
# imports 
from flair.datasets import Corpus
from flair.data_fetcher import NLPTaskDataFetcher

# folder where training and test data are
data_folder = "data/"
# your training file name
train_file = "bilou_training.csv"

# your training file name
test_file = "bilou_test.csv"

columns = {0: "Tokens", 2: "True"}

## Now load our csv into flair corpus
corpus = NLPTaskDataFetcher.load_column_corpus(data_folder,column_format=columns,
                                               train_file=train_file,
                                               test_file=test_file)

print("Done!")
print(corpus.test)

2019-06-17 01:53:16,408 Reading data from data
2019-06-17 01:53:16,411 Train: data/bilou_training.csv
2019-06-17 01:53:16,412 Dev: None
2019-06-17 01:53:16,412 Test: data/bilou_test.csv


/home/anton/Desktop/uni/Resume_NER/nerenv/lib/python3.7/site-packages/wrapt/wrappers.py:564: DeprecationWarning: Call to deprecated function (or staticmethod) load_column_corpus. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  args, kwargs)
/home/anton/Desktop/uni/Resume_NER/nerenv/lib/python3.7/site-packages/wrapt/wrappers.py:564: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  args, kwargs)
/home/anton/Desktop/uni/Resume_NER/nerenv/lib/python3.7/site-packages/wrapt/wrappers.py:564: DeprecationWarning: Call to deprecated function (or staticmethod) read_column_data. (Use 'flair.datasets' instead.) -- Deprecated since version 0.4.1.
  args, kwargs)


Done!
[Sentence: "Dipesh Gulati Co - coordinator of CODE HUNTER at CGC FEST Delhi , Delhi - Email me on Indeed : indeed.com/r/Dipesh-Gulati/17a483e9e19f9106 To work in challenging environment that will provide opportunities for learning and growth with the best efforts for achieving organizational goals and at the same time having self - development through learning , exposure and seeking my own identity " " WORK EXPERIENCE Co - coordinator of CODE HUNTER at CGC FEST ORACLE SQL - 2015 to 2015 EDUCATION MCA Punjab Technical University - Chandigarh , Chandigarh 2017 BCA Bharati Vidyapeeth University - Pune , Maharashtra 2014 CBSE Crescent Public School - New Delhi , Delhi 2011 CBSE Crescent Public School - New Delhi , Delhi 2009 SKILLS ACCESS ( Less than 1 year ) , BUYING ( Less than 1 year ) , BUYING / PROCUREMENT ( Less than 1 year ) , CSS ( Less than 1 year ) , DATABASE ( Less than 1 year ) ADDITIONAL INFORMATION I.T Skills https://www.indeed.com/r/Dipesh-Gulati/17a483e9e19f9106?isid=

In [151]:
print(len(corpus.train))

493


If you could load the corpus without error, you are ready to go on to part 3, where we will work with flair nlp!

If you have something like between 5 and 10k training sentences, and roughly a third of that as

In [152]:
print(corpus.train[0])

Sentence: "Shehzad Hamidani Thane , Maharashtra - Email me on Indeed : indeed.com / r / Shehzad- Hamidani/4976b253a25775dc WORK EXPERIENCE Sales Manager Bakery & Confectionery - Mumbai , Maharashtra - January 2017 to Present I work as a sales manager in the above firm EDUCATION HSC SKILLS Computer Basic https://www.indeed.com/r/Shehzad-Hamidani/4976b253a25775dc?isid=rex-download&ikw=download-top&co=IN https://www.indeed.com/r/Shehzad-Hamidani/4976b253a25775dc?isid=rex-download&ikw=download-top&co=IN" - 50 Tokens
